In [ ]:
'''
DKI1 Bundaran HI
file_path = r'C:\Users\user\OneDrive\IPB\Thesis\02. Development\05. Landcover\20251120\bundaranhi\trajectory_group_1.csv'

DKI2 Kelapa Gadaing
file_path = r'C:\Users\user\OneDrive\IPB\Thesis\02. Development\05. Landcover\20251120\kelapagading\trajectory_group_1.csv'

DKI3 Jagakarsa
file_path = r'C:\Users\user\OneDrive\IPB\Thesis\02. Development\05. Landcover\20251120\jagakarsa\trajectory_group_1.csv'

DKI4 Lubang Buaya
file_path = r'C:\Users\user\OneDrive\IPB\Thesis\02. Development\05. Landcover\20251120\lubangbuaya\trajectory_group_1.csv'

DKI5 Kebun Jeruk
file_path = r'C:\Users\user\OneDrive\IPB\Thesis\02. Development\05. Landcover\20251120\kebunjeruk\trajectory_group_1.csv'



'''

'\nadssdas\n'

In [8]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from datetime import timedelta # Tambahkan import ini

def identify_locations(file_path):
    # 1. Load the data
    try:
        df = pd.read_csv(file_path)
        print("Data loaded successfully.")
    except Exception as e:
        print(f"Error loading file: {e}")
        return

    # --- TAMBAHAN: KONVERSI KE WIB (UTC+7) ---
    if 'DATETIME' in df.columns:
        # Konversi string ke format datetime
        df['DATETIME'] = pd.to_datetime(df['DATETIME'])
        # Tambah 7 jam
        df['DATETIME'] = df['DATETIME'] + timedelta(hours=7)
        print("Datetime converted to WIB (UTC+7).")

    # Check for required columns
    if 'LATITUDE' not in df.columns or 'LONGITUDE' not in df.columns:
        print("Error: CSV must contain 'LATITUDE' and 'LONGITUDE' columns.")
        return

    # 2. Initialize Geocoder
    geolocator = Nominatim(user_agent="my_trajectory_analyzer_v1")
    reverse = RateLimiter(geolocator.reverse, min_delay_seconds=1.5)

    print("Identifying locations... this may take a moment.")
    
    # 3. Function to get location name (Tetap sama)
    def get_place_name(lat, lon):
        try:
            location = reverse((lat, lon), language='en', exactly_one=True)
            if location:
                address = location.raw['address']
                place = address.get('city', 
                           address.get('town', 
                               address.get('village', 
                                   address.get('county', 'Unknown Area'))))
                state = address.get('state', address.get('region', ''))
                return f"{place}, {state}"
        except Exception as e:
            return "Ocean/Remote or Error"
        return "Ocean/Remote"

    # 4. Apply to the dataset
    df_subset = df.iloc[::5].copy() 
    df_subset['Location_Name'] = df_subset.apply(
        lambda row: get_place_name(row['LATITUDE'], row['LONGITUDE']), axis=1
    )

    # 5. Display and Save Result
    print("\n--- Trajectory Location Analysis (WIB) ---")
    print(df_subset[['DATETIME', 'LATITUDE', 'LONGITUDE', 'Location_Name']])

    output_path = file_path.replace('.csv', '_with_locations_WIB.csv')
    df_subset.to_csv(output_path, index=False)
    print(f"\nResults saved to: {output_path}")

if __name__ == "__main__":
    file_path = r'C:\Users\user\OneDrive\IPB\Thesis\02. Development\05. Landcover\20251120\kebunjeruk\trajectory_group_1.csv'
    identify_locations(file_path)

Data loaded successfully.
Datetime converted to WIB (UTC+7).
Identifying locations... this may take a moment.

--- Trajectory Location Analysis (WIB) ---
              DATETIME  LATITUDE  LONGITUDE  \
0  2024-11-17 07:00:00    -6.207    106.753   
5  2024-11-17 02:00:00    -6.271    106.842   
10 2024-11-16 21:00:00    -6.175    107.002   
15 2024-11-16 16:00:00    -5.726    106.880   
20 2024-11-16 11:00:00    -5.434    106.726   
25 2024-11-16 06:00:00    -5.445    106.701   
30 2024-11-16 01:00:00    -5.707    106.537   
35 2024-11-15 20:00:00    -5.954    106.307   
40 2024-11-15 15:00:00    -5.743    106.436   
45 2024-11-15 10:00:00    -5.553    106.799   

                              Location_Name  
0                        West Jakarta, Java  
5                       South Jakarta, Java  
10                        Bekasi, West Java  
15  Special Capital Region of Jakarta, Java  
20  Special Capital Region of Jakarta, Java  
25  Special Capital Region of Jakarta, Java  
30  Sp

In [2]:
import pandas as pd
import requests
import time
import urllib3

# Non-aktifkan warning SSL jika sertifikat kantor/kampus bermasalah
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

def get_land_cover_robust(file_path):
    # 1. Load Data
    try:
        df = pd.read_csv(file_path)
        print(f"Data loaded successfully: {len(df)} rows.")
    except Exception as e:
        print(f"Error loading file: {e}")
        return

    # 2. Definisi Kelas & Warna Sentinel-2
    lc_classes = {
        1:  {'Name': 'Water',              'Color': 'Blue (#1A5BAB)'},
        2:  {'Name': 'Trees',              'Color': 'Dark Green (#358221)'},
        4:  {'Name': 'Flooded Vegetation', 'Color': 'Teal (#87D19E)'},
        5:  {'Name': 'Crops',              'Color': 'Yellow (#FFDB5C)'},
        7:  {'Name': 'Built Area',         'Color': 'Red (#ED022A)'},
        8:  {'Name': 'Bare Ground',        'Color': 'Brown/Gray (#EDE9E4)'},
        9:  {'Name': 'Snow/Ice',           'Color': 'White (#F2FAFF)'},
        10: {'Name': 'Clouds',             'Color': 'Light Gray (#C8C8C8)'},
        11: {'Name': 'Rangeland',          'Color': 'Light Green (#C6AD8D)'}
    }

    # URL Endpoint Sentinel-2
    url = "https://ic.imagery1.arcgis.com/arcgis/rest/services/Sentinel2_10m_LandCover/ImageServer/identify"

    # Headers agar dianggap Browser (PENTING)
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Referer': 'https://livingatlas.arcgis.com/'
    }

    print("\nStarting identification... (Please ensure internet connection)")

    lc_names = []
    lc_colors = []

    # 3. Loop Data
    for index, row in df.iterrows():
        lat = row['LATITUDE']
        lon = row['LONGITUDE']

        params = {
            'f': 'json',
            'geometry': f'{lon},{lat}',
            'geometryType': 'esriGeometryPoint',
            'sr': '4326',  # Coordinate system WGS84
            'returnGeometry': 'false',
            'returnCatalogItems': 'false',
            # Mengambil data terbaru, tapi memprioritaskan yang tutupan lahannya jelas
            'mosaicRule': '{"mosaicMethod":"esriMosaicAttribute","sortField":"acquisitionDate","sortValue":"2024/01/01"}'
        }

        success = False
        retries = 0
        
        # Retry logic: Coba 3 kali jika gagal
        while not success and retries < 3:
            try:
                # verify=False untuk bypass masalah SSL di jaringan kampus/kantor
                response = requests.get(url, params=params, headers=headers, timeout=10, verify=False)
                
                if response.status_code == 200:
                    data = response.json()
                    if 'value' in data:
                        pixel_val = int(data['value'])
                        if pixel_val in lc_classes:
                            lc_names.append(lc_classes[pixel_val]['Name'])
                            lc_colors.append(lc_classes[pixel_val]['Color'])
                            success = True
                        else:
                            lc_names.append(f"Class {pixel_val}")
                            lc_colors.append("Unknown")
                            success = True
                    else:
                        # Kadang response 200 tapi isinya error/kosong
                        retries += 1
                else:
                    retries += 1
            except Exception as e:
                # print(f"Connection Error: {e}") # Uncomment untuk debug
                retries += 1
                time.sleep(1) # Tunggu 1 detik sebelum coba lagi

        if not success:
            lc_names.append("Unknown/No Data")
            lc_colors.append("Unknown")

        # Progress bar sederhana
        if index % 5 == 0:
            print(f"Processed row {index+1}/{len(df)}: {lc_names[-1]}")

    # 4. Simpan
    df['LandCover_Class'] = lc_names
    df['LandCover_Color'] = lc_colors

    output_path = file_path.replace('.csv', '_FINAL_LC.csv')
    df.to_csv(output_path, index=False)
    
    print("\n--- DONE ---")
    print(df[['LATITUDE', 'LONGITUDE', 'LandCover_Class', 'LandCover_Color']].head())
    print(f"File saved to: {output_path}")

if __name__ == "__main__":
    # GANTI PATH INI SESUAI LOKASI FILE ANDA
    file_path = r'C:\Users\user\OneDrive\IPB\Thesis\02. Development\05. Landcover\20251120\bundaranhi\trajectory_group_1_with_locations.csv'
    
    get_land_cover_robust(file_path)

Data loaded successfully: 10 rows.

Starting identification... (Please ensure internet connection)
Processed row 1/10: Unknown/No Data
Processed row 6/10: Unknown/No Data

--- DONE ---
   LATITUDE  LONGITUDE  LandCover_Class LandCover_Color
0    -6.195    106.824  Unknown/No Data         Unknown
1    -6.286    107.153  Unknown/No Data         Unknown
2    -6.287    107.172  Unknown/No Data         Unknown
3    -5.937    107.150  Unknown/No Data         Unknown
4    -5.688    107.475  Unknown/No Data         Unknown
File saved to: C:\Users\user\OneDrive\IPB\Thesis\02. Development\05. Landcover\20251120\bundaranhi\trajectory_group_1_with_locations_FINAL_LC.csv


In [3]:
import pandas as pd
import requests
import urllib3
import time

# 1. NON-AKTIFKAN PERINGATAN KEAMANAN (PENTING UNTUK JARINGAN KAMPUS/KANTOR)
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

def get_land_cover_debug(file_path):
    print("--- MEMULAI PROSES LAND COVER (DEBUG MODE) ---")
    
    # 2. Load Data
    try:
        df = pd.read_csv(file_path)
        print(f"File berhasil diload: {len(df)} baris data.")
    except Exception as e:
        print(f"Gagal buka file: {e}")
        return

    # 3. Definisi Kelas & Warna (Sentinel-2 10m)
    lc_classes = {
        1:  {'Name': 'Water',              'Color': 'Blue (#1A5BAB)'},
        2:  {'Name': 'Trees',              'Color': 'Dark Green (#358221)'},
        4:  {'Name': 'Flooded Vegetation', 'Color': 'Teal (#87D19E)'},
        5:  {'Name': 'Crops',              'Color': 'Yellow (#FFDB5C)'},
        7:  {'Name': 'Built Area',         'Color': 'Red (#ED022A)'},
        8:  {'Name': 'Bare Ground',        'Color': 'Brown/Gray (#EDE9E4)'},
        9:  {'Name': 'Snow/Ice',           'Color': 'White (#F2FAFF)'},
        10: {'Name': 'Clouds',             'Color': 'Light Gray (#C8C8C8)'},
        11: {'Name': 'Rangeland',          'Color': 'Light Green (#C6AD8D)'}
    }

    # URL API Esri (Sentinel-2 LULC)
    url = "https://ic.imagery1.arcgis.com/arcgis/rest/services/Sentinel2_10m_LandCover/ImageServer/identify"

    # Headers Lengkap (Agar dikira Browser Chrome)
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Accept': '*/*',
        'Connection': 'keep-alive',
        'Referer': 'https://livingatlas.arcgis.com/'
    }

    # 4. Fungsi Request dengan Print Error
    def fetch_class(lat, lon):
        params = {
            'f': 'json',
            'geometry': f'{lon},{lat}',
            'geometryType': 'esriGeometryPoint',
            'sr': '4326',
            'returnGeometry': 'false',
            'mosaicRule': '{"mosaicMethod":"esriMosaicAttribute","sortField":"acquisitionDate","sortValue":"2024/01/01"}'
        }
        
        try:
            # Timeout 10 detik, VERIFY=FALSE (Kunci bypass SSL)
            response = requests.get(url, params=params, headers=headers, timeout=10, verify=False)
            
            # Cek jika status bukan 200 OK
            if response.status_code != 200:
                print(f"  [!] Server Error: {response.status_code}")
                return None, "Server Error"

            data = response.json()
            
            # Cek apakah ada error dari API Esri sendiri
            if 'error' in data:
                print(f"  [!] API Error: {data['error']['message']}")
                return None, "API Blocked"

            if 'value' in data:
                val = int(data['value'])
                if val in lc_classes:
                    return lc_classes[val]['Name'], lc_classes[val]['Color']
                else:
                    return f"Class {val}", "Unknown Color"
            else:
                return "No Value", "No Data"

        except requests.exceptions.ConnectTimeout:
            print("  [!] Koneksi Timeout (Internet lambat/diblokir)")
            return None, "Timeout"
        except requests.exceptions.SSLError:
            print("  [!] SSL Error (Sertifikat ditolak)")
            return None, "SSL Error"
        except Exception as e:
            print(f"  [!] Error Lain: {e}")
            return None, "Script Error"

    # 5. Loop Data
    names = []
    colors = []
    
    print("\nMencoba mengambil data untuk setiap titik...")
    for index, row in df.iterrows():
        lat = row['LATITUDE']
        lon = row['LONGITUDE']
        
        # Ambil data
        c_name, c_color = fetch_class(lat, lon)
        
        if c_name is None:
            # Jika gagal, coba sekali lagi (Retry)
            time.sleep(1)
            c_name, c_color = fetch_class(lat, lon)
            if c_name is None:
                c_name = "Unknown/Error"
                c_color = "Unknown"

        names.append(c_name)
        colors.append(c_color)

        # Tampilkan progress setiap 1 data agar terpantau
        print(f"Row {index}: {lat}, {lon} -> {c_name}")

    # 6. Simpan Hasil
    df['LandCover_Class'] = names
    df['LandCover_Color'] = colors

    output_path = file_path.replace('.csv', '_DEBUG_RESULT.csv')
    df.to_csv(output_path, index=False)
    print(f"\n--- SELESAI ---\nFile disimpan di: {output_path}")

if __name__ == "__main__":
    # GANTI PATH SESUAI FILE ANDA
    file_path = r'C:\Users\user\OneDrive\IPB\Thesis\02. Development\05. Landcover\20251120\lubangbuaya\trajectory_group_1_with_locations.csv'
    
    get_land_cover_debug(file_path)

--- MEMULAI PROSES LAND COVER (DEBUG MODE) ---
File berhasil diload: 10 baris data.

Mencoba mengambil data untuk setiap titik...
  [!] API Error: Unable to complete operation.
  [!] API Error: Unable to complete operation.
Row 0: -6.289, 106.909 -> Unknown/Error
  [!] API Error: Unable to complete operation.
  [!] API Error: Unable to complete operation.
Row 1: -6.467, 107.23 -> Unknown/Error
  [!] API Error: Unable to complete operation.
  [!] API Error: Unable to complete operation.
Row 2: -6.789, 107.665 -> Unknown/Error
  [!] API Error: Unable to complete operation.
  [!] API Error: Unable to complete operation.
Row 3: -6.805, 108.106 -> Unknown/Error
  [!] API Error: Unable to complete operation.
  [!] API Error: Unable to complete operation.
Row 4: -6.624, 108.717 -> Unknown/Error
  [!] API Error: Unable to complete operation.
  [!] API Error: Unable to complete operation.
Row 5: -7.083, 108.83 -> Unknown/Error
  [!] API Error: Unable to complete operation.
  [!] API Error: Unab

In [4]:
import pandas as pd
import requests
import time
import json

def get_land_cover_osm(file_path):
    print("--- STARTING OSM LAND COVER ANALYSIS ---")
    
    # 1. Load Data
    try:
        df = pd.read_csv(file_path)
        print(f"Data Loaded: {len(df)} rows.")
    except Exception as e:
        print(f"Error loading file: {e}")
        return

    # 2. Definisi Mapping (OSM Tags -> Warna Land Cover)
    def map_osm_to_color(tags):
        # Gabungkan semua value tags menjadi satu string agar mudah dicek
        tag_str = " ".join(str(v) for v in tags.values()).lower()
        
        # Prioritas Pengecekan (Urutan itu penting!)
        
        # 1. Built Area (Red)
        if any(x in tag_str for x in ['residential', 'commercial', 'industrial', 'retail', 'building', 'house', 'school', 'hospital', 'village', 'town']):
            return 'Built Area', 'Red (#ED022A)'
            
        # 2. Water (Blue)
        if any(x in tag_str for x in ['water', 'river', 'reservoir', 'pond', 'lake', 'coastline']):
            return 'Water', 'Blue (#1A5BAB)'
            
        # 3. Crops (Yellow)
        if any(x in tag_str for x in ['farm', 'farmland', 'orchard', 'vineyard', 'paddy']):
            return 'Crops', 'Yellow (#FFDB5C)'
            
        # 4. Trees (Dark Green)
        if any(x in tag_str for x in ['forest', 'wood', 'jungle']):
            return 'Trees', 'Dark Green (#358221)'
            
        # 5. Rangeland/Grass (Light Green)
        if any(x in tag_str for x in ['grass', 'meadow', 'scrub', 'heath', 'park', 'garden', 'pitch']):
            return 'Rangeland', 'Light Green (#C6AD8D)'
            
        # Default jika tidak ada info spesifik (anggap tanah terbuka/umum)
        return 'Bare/Mixed', 'Brown/Gray (#EDE9E4)'

    # 3. Fungsi Request ke Overpass API
    def fetch_osm_data(lat, lon):
        overpass_url = "http://overpass-api.de/api/interpreter"
        # Query mencari way/relation (area) di sekitar 50m dari titik
        overpass_query = f"""
        [out:json][timeout:10];
        (
          way(around:50, {lat}, {lon})["landuse"];
          way(around:50, {lat}, {lon})["natural"];
          relation(around:50, {lat}, {lon})["landuse"];
          relation(around:50, {lat}, {lon})["natural"];
        );
        out tags;
        """
        try:
            response = requests.get(overpass_url, params={'data': overpass_query}, timeout=15)
            if response.status_code == 200:
                data = response.json()
                if len(data['elements']) > 0:
                    # Ambil tags dari elemen pertama yang ditemukan
                    return data['elements'][0]['tags']
            return None
        except Exception:
            return None

    # 4. Loop Data
    lc_classes = []
    lc_colors = []

    print("\nProcessing locations (This checks public maps, please wait)...")
    
    for index, row in df.iterrows():
        lat = row['LATITUDE']
        lon = row['LONGITUDE']
        
        # Coba ambil data OSM
        tags = fetch_osm_data(lat, lon)
        
        if tags:
            c_class, c_color = map_osm_to_color(tags)
        else:
            # Jika tidak ada data area spesifik di OSM, kita gunakan logika sederhana
            # Jika di Jakarta/Jawa (populasi padat) -> asumsi 'Built/Crops' campuran
            # Tapi biar aman kita set "Unknown/Open"
            c_class = "Open/Unclassified" 
            c_color = "Brown/Gray (#EDE9E4)"
            
        lc_classes.append(c_class)
        lc_colors.append(c_color)

        # Progress bar
        print(f"Row {index}: {c_class}")
        
        # Jeda 1 detik agar tidak diblokir server OSM (PENTING!)
        time.sleep(1)

    # 5. Save
    df['LandCover_Class'] = lc_classes
    df['LandCover_Color'] = lc_colors

    output_path = file_path.replace('.csv', '_OSM_LC.csv')
    df.to_csv(output_path, index=False)
    
    print("\n--- DONE ---")
    print(df[['LATITUDE', 'LONGITUDE', 'LandCover_Class', 'LandCover_Color']].head())
    print(f"File saved to: {output_path}")

if __name__ == "__main__":
    # GANTI PATH FILE ANDA DISINI
    file_path = r'C:\Users\user\OneDrive\IPB\Thesis\02. Development\05. Landcover\20251120\lubangbuaya\trajectory_group_1_with_locations.csv'
    
    get_land_cover_osm(file_path)

--- STARTING OSM LAND COVER ANALYSIS ---
Data Loaded: 10 rows.

Processing locations (This checks public maps, please wait)...
Row 0: Open/Unclassified
Row 1: Open/Unclassified
Row 2: Open/Unclassified
Row 3: Open/Unclassified
Row 4: Open/Unclassified
Row 5: Open/Unclassified
Row 6: Open/Unclassified
Row 7: Open/Unclassified
Row 8: Open/Unclassified
Row 9: Open/Unclassified

--- DONE ---
   LATITUDE  LONGITUDE    LandCover_Class       LandCover_Color
0    -6.289    106.909  Open/Unclassified  Brown/Gray (#EDE9E4)
1    -6.467    107.230  Open/Unclassified  Brown/Gray (#EDE9E4)
2    -6.789    107.665  Open/Unclassified  Brown/Gray (#EDE9E4)
3    -6.805    108.106  Open/Unclassified  Brown/Gray (#EDE9E4)
4    -6.624    108.717  Open/Unclassified  Brown/Gray (#EDE9E4)
File saved to: C:\Users\user\OneDrive\IPB\Thesis\02. Development\05. Landcover\20251120\lubangbuaya\trajectory_group_1_with_locations_OSM_LC.csv


In [25]:
import pandas as pd
import requests
import time
import json

def get_land_cover_osm(file_path):
    print("--- STARTING OSM LAND COVER ANALYSIS ---")
    
    # 1. Load Data
    try:
        df = pd.read_csv(file_path)
        print(f"Data Loaded: {len(df)} rows.")
    except Exception as e:
        print(f"Error loading file: {e}")
        return

    # 2. Definisi Mapping (OSM Tags -> Warna Land Cover)
    def map_osm_to_color(tags):
        # Gabungkan semua value tags menjadi satu string agar mudah dicek
        tag_str = " ".join(str(v) for v in tags.values()).lower()
        
        # Prioritas Pengecekan (Urutan itu penting!)
        
        # 1. Built Area (Red)
        if any(x in tag_str for x in ['residential', 'commercial', 'industrial', 'retail', 'building', 'house', 'school', 'hospital', 'village', 'town']):
            return 'Built Area', 'Red (#ED022A)'
            
        # 2. Water (Blue)
        if any(x in tag_str for x in ['water', 'river', 'reservoir', 'pond', 'lake', 'coastline']):
            return 'Water', 'Blue (#1A5BAB)'
            
        # 3. Crops (Yellow)
        if any(x in tag_str for x in ['farm', 'farmland', 'orchard', 'vineyard', 'paddy']):
            return 'Crops', 'Yellow (#FFDB5C)'
            
        # 4. Trees (Dark Green)
        if any(x in tag_str for x in ['forest', 'wood', 'jungle']):
            return 'Trees', 'Dark Green (#358221)'
            
        # 5. Rangeland/Grass (Light Green)
        if any(x in tag_str for x in ['grass', 'meadow', 'scrub', 'heath', 'park', 'garden', 'pitch']):
            return 'Rangeland', 'Light Green (#C6AD8D)'
            
        # Default jika tidak ada info spesifik (anggap tanah terbuka/umum)
        return 'Bare/Mixed', 'Brown/Gray (#EDE9E4)'

    # 3. Fungsi Request ke Overpass API
    def fetch_osm_data(lat, lon):
        overpass_url = "http://overpass-api.de/api/interpreter"
        # Query mencari way/relation (area) di sekitar 50m dari titik
        overpass_query = f"""
        [out:json][timeout:10];
        (
          way(around:50, {lat}, {lon})["landuse"];
          way(around:50, {lat}, {lon})["natural"];
          relation(around:50, {lat}, {lon})["landuse"];
          relation(around:50, {lat}, {lon})["natural"];
        );
        out tags;
        """
        try:
            response = requests.get(overpass_url, params={'data': overpass_query}, timeout=15)
            if response.status_code == 200:
                data = response.json()
                if len(data['elements']) > 0:
                    # Ambil tags dari elemen pertama yang ditemukan
                    return data['elements'][0]['tags']
            return None
        except Exception:
            return None

    # 4. Loop Data
    lc_classes = []
    lc_colors = []

    print("\nProcessing locations (This checks public maps, please wait)...")
    
    for index, row in df.iterrows():
        lat = row['LATITUDE']
        lon = row['LONGITUDE']
        
        # Coba ambil data OSM
        tags = fetch_osm_data(lat, lon)
        
        if tags:
            c_class, c_color = map_osm_to_color(tags)
        else:
            # Jika tidak ada data area spesifik di OSM, kita gunakan logika sederhana
            # Jika di Jakarta/Jawa (populasi padat) -> asumsi 'Built/Crops' campuran
            # Tapi biar aman kita set "Unknown/Open"
            c_class = "Open/Unclassified" 
            c_color = "Brown/Gray (#EDE9E4)"
            
        lc_classes.append(c_class)
        lc_colors.append(c_color)

        # Progress bar
        print(f"Row {index}: {c_class}")
        
        # Jeda 1 detik agar tidak diblokir server OSM (PENTING!)
        time.sleep(1)

    # 5. Save
    df['LandCover_Class'] = lc_classes
    df['LandCover_Color'] = lc_colors

    output_path = file_path.replace('.csv', '_OSM_LC.csv')
    df.to_csv(output_path, index=False)
    
    print("\n--- DONE ---")
    print(df[['LATITUDE', 'LONGITUDE', 'LandCover_Class', 'LandCover_Color']].head())
    print(f"File saved to: {output_path}")

if __name__ == "__main__":
    # GANTI PATH FILE ANDA DISINI
    file_path = r'C:\Users\user\OneDrive\IPB\Thesis\02. Development\05. Landcover\20251120\lubangbuaya\trajectory_group_1_with_locations.csv'
    
    get_land_cover_osm(file_path)

--- STARTING OSM LAND COVER ANALYSIS ---
Data Loaded: 10 rows.

Processing locations (This checks public maps, please wait)...
Row 0: Open/Unclassified
Row 1: Open/Unclassified
Row 2: Open/Unclassified
Row 3: Open/Unclassified
Row 4: Open/Unclassified
Row 5: Open/Unclassified
Row 6: Open/Unclassified
Row 7: Open/Unclassified
Row 8: Open/Unclassified
Row 9: Open/Unclassified

--- DONE ---
   LATITUDE  LONGITUDE    LandCover_Class       LandCover_Color
0    -6.289    106.909  Open/Unclassified  Brown/Gray (#EDE9E4)
1    -6.467    107.230  Open/Unclassified  Brown/Gray (#EDE9E4)
2    -6.789    107.665  Open/Unclassified  Brown/Gray (#EDE9E4)
3    -6.805    108.106  Open/Unclassified  Brown/Gray (#EDE9E4)
4    -6.624    108.717  Open/Unclassified  Brown/Gray (#EDE9E4)
File saved to: C:\Users\user\OneDrive\IPB\Thesis\02. Development\05. Landcover\20251120\lubangbuaya\trajectory_group_1_with_locations_OSM_LC.csv
